# Interval method for weather predictions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import mean_squared_error

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 12)

In [4]:
def relative_rmse(target, preds):
    rmse = mean_squared_error(target, preds, squared=False)
    avg = np.full(len(preds), preds.mean())
    const_rmse = mean_squared_error(target, avg, squared=False)
    return rmse / const_rmse

### Reading data

In [5]:
high = pd.read_csv('predicted_weather/prophet_high.csv')
low = pd.read_csv('predicted_weather/prophet_low.csv')

In [6]:
low['VAP'][low['VAP'] < 0] = 0.06
low['RAIN'][low['RAIN'] < 0] = 0

/var/folders/1f/xyznj37x1n337kxt0v5533rw0000gn/T/ipykernel_13481/844755778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low['VAP'][low['VAP'] < 0] = 0.06
/var/folders/1f/xyznj37x1n337kxt0v5533rw0000gn/T/ipykernel_13481/844755778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low['RAIN'][low['RAIN'] < 0] = 0


In [7]:
high['VAP'][high['VAP'] < 0] = 0.06
high['RAIN'][high['RAIN'] < 0] = 0

/var/folders/1f/xyznj37x1n337kxt0v5533rw0000gn/T/ipykernel_13481/2604046005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high['VAP'][high['VAP'] < 0] = 0.06
/var/folders/1f/xyznj37x1n337kxt0v5533rw0000gn/T/ipykernel_13481/2604046005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high['RAIN'][high['RAIN'] < 0] = 0


In [8]:
low.to_csv('predicted_weather/prophet_low.csv', index=False)
high.to_csv('predicted_weather/prophet_high.csv', index=False)

In [9]:
pivot = low.merge(high, on=['DAY', 'SNOWDEPTH'])

In [10]:
def get_dateset(df, name, n):
    df[f'Width_{name}'] = df[f'{name}_y'] - df[f'{name}_x'] 
    df[f'Delta_{name}'] = df[f'Width_{name}'] / n
    temp = df[[f'Delta_{name}', f'{name}_x']].copy()
    for i in range(1, n+1):
        temp[f'{name}_{i}'] = temp[f'{name}_x'] + i * temp[f'Delta_{name}']
    temp.rename({f'{name}_x': f'{name}_0'}, axis=1, inplace=True)
    return temp.drop([f'Delta_{name}'], axis=1)

In [11]:
n = 8

In [12]:
irrad = get_dateset(pivot, 'IRRAD', n)
tmin = get_dateset(pivot, 'TMIN', n)
tmax = get_dateset(pivot, 'TMAX', n)
vap = get_dateset(pivot, 'VAP', n)
wind = get_dateset(pivot, 'WIND', n)
rain = get_dateset(pivot, 'RAIN', n)

In [13]:
irrad.to_csv('predicted_weather/interval_data/irrad.csv', index=False)
tmin.to_csv('predicted_weather/interval_data/tmin.csv', index=False)
tmax.to_csv('predicted_weather/interval_data/tmax.csv', index=False)
vap.to_csv('predicted_weather/interval_data/vap.csv', index=False)
wind.to_csv('predicted_weather/interval_data/wind.csv', index=False)
rain.to_csv('predicted_weather/interval_data/rain.csv', index=False)

In [24]:
def product_columns(x:int):
    all_files = []
    for x1 in range(n+1):
        for x2 in range(n+1):
            for x3 in range(n+1):
                for x4 in range(n+1):
                    for x5 in range(n+1):
                        for x6 in range(n+1):
                            tmp = low[['DAY', 'SNOWDEPTH']].copy()
                            tmp['IRRAD'] = irrad[f'IRRAD_{x1}']
                            tmp['TMIN'] = tmin[f'TMIN_{x2}']
                            tmp['TMAX'] = tmax[f'TMAX_{x3}']
                            tmp['VAP'] = vap[f'VAP_{x4}']
                            tmp['WIND'] = wind[f'WIND_{x5}']
                            tmp['RAIN'] = rain[f'RAIN_{x6}']
                            fname = f'predicted_weather/interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv'
                            fname = f'predicted_weather/interval_data/{x}_{x2}_{x3}_{x4}_{x5}_{x6}.csv'
                            all_files.append(fname)
        break
                            # tmp.to_csv(fname, index=False)
    return all_files
                            # Думаю хорошо бы удалять CSV с погодой после запуска - os.remove(fname)

In [65]:
def product_columns(n:int, x1:int, x2:int):
    all_files = []
    for x3 in range(n+1):
        for x4 in range(n+1):
            for x5 in range(n+1):
                for x6 in range(n+1):
                    # tmp = low[['DAY', 'SNOWDEPTH']].copy()
                    # tmp['IRRAD'] = irrad[f'IRRAD_{x1}']
                    # tmp['TMIN'] = tmin[f'TMIN_{x2}']
                    # tmp['TMAX'] = tmax[f'TMAX_{x3}']
                    # tmp['VAP'] = vap[f'VAP_{x4}']
                    # tmp['WIND'] = wind[f'WIND_{x5}']
                    # tmp['RAIN'] = rain[f'RAIN_{x6}']
                    # fname = f'predicted_weather/interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv'
                    fname = f'predicted_weather/interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv'
                    all_files.append(fname)
        

                            # tmp.to_csv(fname, index=False)
    return all_files
                            # Думаю хорошо бы удалять CSV с погодой после запуска - os.remove(fname)
all_files = product_columns(n=8, x1=0, x2=1)

## WOFOST & Intervals

In [ ]:
import numpy as np
import os, sys, math, yaml
import datetime as dt
from dateutil.parser import parse
import pandas as pd
#Plotting
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt
import json

# from pcse.fileinput import YAMLAgroManagementReader
from pcse.util import WOFOST71SiteDataProvider, WOFOST72SiteDataProvider
# from pcse.fileinput import PCSEFileReader

import pcse

from pcse.models import Wofost72_PP, Wofost72_WLP_FD
from pcse.fileinput import YAMLCropDataProvider

from pcse.util import WOFOST72SiteDataProvider, DummySoilDataProvider
from pcse.base import ParameterProvider
from pcse.engine import Engine
from pcse.fileinput.csvweatherdataprovider import CSVWeatherDataProvider


def prepareWeather(in_fname: str):
    """
    Read weather file, after convert to WOFOST CSV format and 
    read into memory to PCSE weather Class
    
    in: in_fname (str) - path to csv file with weather time-series
    """
    # read weather time-series
    weather_df = pd.read_csv(in_fname)
    # future name

    path_to_save_csv_file = os.path.splitext(in_fname)[0]+'_WOFOST.csv'
    # path_to_save_csv_file = './wofost_weather.csv'
    # pattern for WOFOST format
    text = open('./pattern.csv', "r")
    dictReplace = {
        "1111": "37.0",
        "2222": "51.5",
        "3333": "210.05", # m
        "4444": "0.172729",
        "5555": "0.565318",
    }
    for key, value in dictReplace.items():
        text = "".join([i for i in text]).replace(key, str(value))
    x = open(path_to_save_csv_file, "w")
    x.writelines(text)
    x.close()
    weather_df.to_csv(
        path_to_save_csv_file, mode="a", header=False, index=False, na_rep='NaN'
    )
    weather = CSVWeatherDataProvider(path_to_save_csv_file)
    return weather


def run_wofost(weather, 
              sowing_date:str,
              harvesting_date:str,
              crop_end_type: str = 'maturity',
              crop_name:str = 'sugarbeet',
              crop_variety:str = 'Sugarbeet_603', 
              models: list = ['FLD', 'PP'])-> dict:

    """
        Prepare parameters: meteo, agrotechnology and run WOFOST simulation in PP and FLD modes.   

        Parameters
        ----------
        crop_name: str
            Crop name fow WOFOST model
        crop_variety: str
            Crop variety fow WOFOST crop 
        sowing_date: str
            Date of crop sowing, ex. %Y-%m-%d
        harvest_date: str 
        - optional
            Date of crop harvest, ex. %Y-%m-%d
            
        Returns
        -------
        results: dict 
            dict with PCSE objects: cropd, wdp
            if run_similation: True 
                return computed default yield
        To-Do
        -----


        Example
        -------

    """
    cropd = YAMLCropDataProvider(repository="https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/")
    # Some site parameters
    sited = WOFOST71SiteDataProvider(WAV=50, SMLIM=0.7)

    soild = DummySoilDataProvider() # Used real soil parameters from kshen 

    soild['SMW'] = 0.15
    soild['SMFCF'] = 0.3
    soild['SM0'] = 0.566 

    _campaign_start_date_dt = parse(sowing_date) -  dt.timedelta(days=5)
    campaign_start_date = dt.datetime.strftime(_campaign_start_date_dt, format='%Y-%m-%d')

    max_duration = 300

    # Here we define the agromanagement for crop
    agro_yaml = f"""
    - {campaign_start_date}:
        CropCalendar:
            crop_name: {crop_name}
            variety_name: {crop_variety}
            crop_start_date: {sowing_date}
            crop_start_type: emergence
            crop_end_date: {harvesting_date}
            crop_end_type: {crop_end_type}
            max_duration: {max_duration}
        TimedEvents: null
        StateEvents: null
    """
    agro = yaml.safe_load(agro_yaml)

    firstkey = list(agro[0])[0]
    cropcalendar = agro[0][firstkey]['CropCalendar'] 

    cropd.set_active_crop(cropcalendar['crop_name'], cropcalendar['variety_name'])

    params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

    defualtCropYield={}
    
    modelDict = {'FLD':Wofost72_WLP_FD, 
                'PP': Wofost72_PP}
    for model_type in ['FLD']:
        params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)
        model_runner = modelDict[model_type](params, weather, agro)
        model_runner.run_till_terminate()
        r = model_runner.get_output()
        defualtCropYield[model_type]=r
        del model_runner
    del weather
    return defualtCropYield


def getCropCalendar(crop:str,
                    year:str)->dict:

    dictAgro = {'barley': {'plant_day': '2021-04-30', 'harvest_day': '2021-09-06'},
    'soybean': {'plant_day': '2021-04-15', 'harvest_day': '2021-08-16'},
    'wheat': {'plant_day': '2021-05-18', 'harvest_day': '2021-09-01'},
    'sugarbeet': {'plant_day': '2021-04-28', 'harvest_day': '2021-10-05'}}

    return {'plant_day': dictAgro[crop]['plant_day'].replace('2021', year),
            'harvest_day': dictAgro[crop]['harvest_day'].replace('2021', year)}



cropsDict = {'barley':'Spring_barley_301', 
            'soybean':'Soybean_901',
            'sunflower':'Sunflower_1101',
            'maize':'Grain_maize_201', 
            'sugarbeet':'Sugarbeet_603'}
            

cols = ['crop', 'year', 'WOFOST_FLD']
crops = ['barley', 'soybean', 'sugarbeet']



In [87]:
def computeCrop(general_df: pd.DataFrame, weather_fname: str, uuid_code:str):
    cols = ['crop', 'year', 'WOFOST_FLD', 'weather_uuid']
    weather = prepareWeather(weather_fname)
    df = pd.DataFrame(columns=cols)

    for crop in crops:
        for year in range(2015,2020):
            cropCal = getCropCalendar(crop, str(year))
            crop_model_yield = run_wofost(weather = weather,
                    sowing_date = cropCal['plant_day'],
                    harvesting_date = cropCal['harvest_day'],
                    crop_name = crop,
                    crop_variety = cropsDict[crop],
                    crop_end_type='harvest')
            water_limited_df = pd.DataFrame(crop_model_yield['FLD'])
            water_limited_yield = crop_model_yield['FLD'][-1]['TWSO']
            df.loc[len(df)] = [crop, year, water_limited_yield, uuid_code]
    return pd.concat([general_df,df])

In [ ]:
from operator import ge


def product_columns(n: int, x1: int, x2: int):
    cols = ["crop", "year", "WOFOST_FLD", "weather_uuid"]
    general_df = pd.DataFrame(columns=cols)

    all_files = []
    for x3 in range(n + 1):
        for x4 in range(n + 1):
            for x5 in range(n + 1):
                for x6 in range(n + 1):
                    tmp = low[["DAY", "SNOWDEPTH"]].copy()
                    tmp["IRRAD"] = irrad[f"IRRAD_{x1}"]
                    tmp["TMIN"] = tmin[f"TMIN_{x2}"]
                    tmp["TMAX"] = tmax[f"TMAX_{x3}"]
                    tmp["VAP"] = vap[f"VAP_{x4}"]
                    tmp["WIND"] = wind[f"WIND_{x5}"]
                    tmp["RAIN"] = rain[f"RAIN_{x6}"]
                    fname = f"predicted_weather/interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv"

                    weather_uuid = f"{x1}_{x2}_{x3}_{x4}_{x5}_{x6}"
                    tmp.to_csv(fname, index=False)
                    all_files.append(fname)
                    general_df = computeCrop(
                        general_df=general_df,
                        weather_fname=fname,
                        uuid_code=weather_uuid,
                    )
                    os.remove(fname)
                    # save data

                    if len(all_files)%1000==0:
                        dirname = './results/'
                        general_fname = os.path.join(dirname, f"WOFOST_{x1}_{x2}.csv")
                        general_df.to_csv(general_fname, index=False)
                    # tmp.to_csv(fname, index=False)
    return all_files
    # Думаю хорошо бы удалять CSV с погодой после запуска - os.remove(fname)


all_files = product_columns(n=8, x1=0, x2=1)


In [159]:
import numpy as np
import os, yaml
import datetime as dt
from dateutil.parser import parse
import pandas as pd
import logging

# Plotting
import datetime as dt


import pcse
from pcse.util import WOFOST71SiteDataProvider, WOFOST72SiteDataProvider
from pcse.models import Wofost72_PP, Wofost72_WLP_FD
from pcse.fileinput import YAMLCropDataProvider

from pcse.util import WOFOST72SiteDataProvider, DummySoilDataProvider
from pcse.base import ParameterProvider
from pcse.engine import Engine
from pcse.fileinput.csvweatherdataprovider import CSVWeatherDataProvider


def prepareWeather(in_fname: str):
    """
    Read weather file, after convert to WOFOST CSV format and
    read into memory to PCSE weather Class

    in: in_fname (str) - path to csv file with weather time-series
    """
    # read weather time-series
    weather_df = pd.read_csv(in_fname)
    # future name

    path_to_save_csv_file = os.path.splitext(in_fname)[0] + "_WOFOST.csv"
    # path_to_save_csv_file = './wofost_weather.csv'
    # pattern for WOFOST format
    text = open("./pattern.csv", "r")
    dictReplace = {
        "1111": "37.0",
        "2222": "51.5",
        "3333": "210.05",  # m
        "4444": "0.172729",
        "5555": "0.565318",
    }
    for key, value in dictReplace.items():
        text = "".join([i for i in text]).replace(key, str(value))
    x = open(path_to_save_csv_file, "w")
    x.writelines(text)
    x.close()
    weather_df.to_csv(
        path_to_save_csv_file, mode="a", header=False, index=False, na_rep="NaN"
    )
    weather = CSVWeatherDataProvider(path_to_save_csv_file)
    os.remove(path_to_save_csv_file)
    return weather


def run_wofost(
    weather,
    sowing_date: str,
    harvesting_date: str,
    crop_end_type: str = "maturity",
    crop_name: str = "sugarbeet",
    crop_variety: str = "Sugarbeet_603",
    models: list = ["FLD", "PP"],
) -> dict:

    """
    Prepare parameters: meteo, agrotechnology and run WOFOST simulation in PP and FLD modes.

    Parameters
    ----------
    crop_name: str
        Crop name fow WOFOST model
    crop_variety: str
        Crop variety fow WOFOST crop
    sowing_date: str
        Date of crop sowing, ex. %Y-%m-%d
    harvest_date: str
    - optional
        Date of crop harvest, ex. %Y-%m-%d

    Returns
    -------
    results: dict
        dict with PCSE objects: cropd, wdp
        if run_similation: True
            return computed default yield
    To-Do
    -----


    Example
    -------

    """
    cropd = YAMLCropDataProvider(
        repository="https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
    )
    # Some site parameters
    sited = WOFOST71SiteDataProvider(WAV=50, SMLIM=0.7)

    soild = DummySoilDataProvider()  # Used real soil parameters from kshen

    soild["SMW"] = 0.15
    soild["SMFCF"] = 0.3
    soild["SM0"] = 0.566

    _campaign_start_date_dt = parse(sowing_date) - dt.timedelta(days=5)
    campaign_start_date = dt.datetime.strftime(
        _campaign_start_date_dt, format="%Y-%m-%d"
    )

    max_duration = 300

    # Here we define the agromanagement for crop
    agro_yaml = f"""
    - {campaign_start_date}:
        CropCalendar:
            crop_name: {crop_name}
            variety_name: {crop_variety}
            crop_start_date: {sowing_date}
            crop_start_type: emergence
            crop_end_date: {harvesting_date}
            crop_end_type: {crop_end_type}
            max_duration: {max_duration}
        TimedEvents: null
        StateEvents: null
    """
    agro = yaml.safe_load(agro_yaml)

    firstkey = list(agro[0])[0]
    cropcalendar = agro[0][firstkey]["CropCalendar"]

    cropd.set_active_crop(cropcalendar["crop_name"], cropcalendar["variety_name"])

    params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

    defualtCropYield = {}

    modelDict = {"FLD": Wofost72_WLP_FD, "PP": Wofost72_PP}
    for model_type in ["FLD"]:
        params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)
        model_runner = modelDict[model_type](params, weather, agro)
        model_runner.run_till_terminate()
        r = model_runner.get_output()
        defualtCropYield[model_type] = r
        del model_runner
    del weather
    return defualtCropYield


def getCropCalendar(crop: str, year: str) -> dict:

    dictAgro = {
        "barley": {"plant_day": "2021-04-30", "harvest_day": "2021-09-06"},
        "soybean": {"plant_day": "2021-04-15", "harvest_day": "2021-08-16"},
        "wheat": {"plant_day": "2021-05-18", "harvest_day": "2021-09-01"},
        "sugarbeet": {"plant_day": "2021-04-28", "harvest_day": "2021-10-05"},
    }

    return {
        "plant_day": dictAgro[crop]["plant_day"].replace("2021", year),
        "harvest_day": dictAgro[crop]["harvest_day"].replace("2021", year),
    }


def computeCrop(general_df: pd.DataFrame, weather_fname: str, uuid_code: str):
    cols = ["crop", "year", "WOFOST_FLD", "weather_uuid"]
    weather = prepareWeather(weather_fname)
    df = pd.DataFrame(columns=cols)

    for crop in crops:
        for year in range(2015, 2020):
            cropCal = getCropCalendar(crop, str(year))
            crop_model_yield = run_wofost(
                weather=weather,
                sowing_date=cropCal["plant_day"],
                harvesting_date=cropCal["harvest_day"],
                crop_name=crop,
                crop_variety=cropsDict[crop],
                crop_end_type="harvest",
            )
            water_limited_df = pd.DataFrame(crop_model_yield["FLD"])
            water_limited_yield = crop_model_yield["FLD"][-1]["TWSO"]
            df.loc[len(df)] = [crop, year, water_limited_yield, uuid_code]
    return pd.concat([general_df, df])


cropsDict = {
    "barley": "Spring_barley_301",
    "soybean": "Soybean_901",
    "sunflower": "Sunflower_1101",
    "maize": "Grain_maize_201",
    "sugarbeet": "Sugarbeet_603",
}


cols = ["crop", "year", "WOFOST_FLD"]
crops = ["barley", "soybean", "sugarbeet"]


def prepare_datasets(dirname: str):
    irrad = pd.read_csv("predicted_weather/interval_data/irrad.csv")
    tmin = pd.read_csv("predicted_weather/interval_data/tmin.csv")
    tmax = pd.read_csv("predicted_weather/interval_data/tmax.csv")
    vap = pd.read_csv("predicted_weather/interval_data/vap.csv")
    wind = pd.read_csv("predicted_weather/interval_data/wind.csv")
    rain = pd.read_csv("predicted_weather/interval_data/rain.csv")
    low = pd.read_csv("predicted_weather/prophet_low.csv")


def checkIrrad(df: pd.DataFrame)->pd.DataFrame:
    df = df.applymap(lambda x: 0 if x<0 else x)
    df = df.applymap(lambda x: 40000000-1 if x>40000000 else x)
    return df



def checkVAP(df: pd.DataFrame)->pd.DataFrame:
    df = df.applymap(lambda x: 0.07 if x<0.06 else x)
    df = df.applymap(lambda x: 199.3-1 if x>199.3 else x)
    return df

def product_columns(n: int, x1: int, x2: int):
    logging.info(f"Compute weathe scenarios: {x1} {x2}")
    cols = ["crop", "year", "WOFOST_FLD", "weather_uuid"]
    general_df = pd.DataFrame(columns=cols)
    dirname = "predicted_weather"
    logging.info("Read weather files")
    irrad = pd.read_csv(os.path.join(dirname, "interval_data/irrad.csv"))
    irrad  = checkIrrad(irrad)
    tmin = pd.read_csv(os.path.join(dirname, "interval_data/tmin.csv"))
    tmax = pd.read_csv(os.path.join(dirname, "interval_data/tmax.csv"))
    vap = pd.read_csv(os.path.join(dirname, "interval_data/vap.csv"))
    vap = checkVAP(vap)
    wind = pd.read_csv(os.path.join(dirname, "interval_data/wind.csv"))
    rain = pd.read_csv(os.path.join(dirname, "interval_data/rain.csv"))
    low = pd.read_csv(os.path.join(dirname, "prophet_low.csv"))
    for x3 in range(n + 1):

        for x4 in range(n + 1):
            for x5 in range(n + 1):
                for x6 in range(n + 1):
                    tmp = low[["DAY"]].copy()
                    tmp["IRRAD"] = irrad[f"IRRAD_{x1}"]
                    tmp["TMIN"] = tmin[f"TMIN_{x2}"]
                    tmp["TMAX"] = tmax[f"TMAX_{x3}"]
                    tmp["VAP"] = vap[f"VAP_{x4}"]
                    tmp["WIND"] = wind[f"WIND_{x5}"]
                    tmp["RAIN"] = rain[f"RAIN_{x6}"]
                    tmp["SNOWDEPTH"] = low[["SNOWDEPTH"]]
                    fname = os.path.join(dirname, f"interval_data/{x1}_{x2}_{x3}_{x4}_{x5}_{x6}.csv")

                    weather_uuid = f"{x1}_{x2}_{x3}_{x4}_{x5}_{x6}"
                    tmp.to_csv(fname, index=False)
        
                    general_df = computeCrop(
                        general_df=general_df,
                        weather_fname=fname,
                        uuid_code=weather_uuid,
                    )
                    os.remove(fname)
                    # save data
                    if len(general_df) % 1000 == 0:
                        dirname = "./results/"
                        general_fname = os.path.join(dirname, f"WOFOST_{x1}_{x2}.csv")
                        general_df.to_csv(general_fname, index=False)

    return general_df



def main(x1: int, x2: int):
    all_files = product_columns(n=8, x1=0, x2=1)


if __name__ == "__main__":
    import argparse

    logging.basicConfig(
        format="- %(asctime)s - %(levelname)s - %(message)s",
        level=logging.INFO,
        datefmt="%H:%M:%S",
    )
    parser = argparse.ArgumentParser()

    parser.add_argument("--x1", default=0, type=int)
    parser.add_argument("--x2", default=0, type=int)
    args = parser.parse_args()
    main(x1=args.x1, x2=args.x2)


In [160]:
main(x1=0, x2=1)

Exception ignored in: <function connect.<locals>.remove at 0x13e1b9790>
Traceback (most recent call last):
  File "/Users/mikhailgasanov/geo/lib/python3.8/site-packages/pcse/pydispatch/dispatcher.py", line 144, in remove
    def remove(object, senderkey=senderkey):
KeyboardInterrupt: 


In [154]:
df.applymap(lambda x: 0 if x<0 else x)

,IRRAD_0,IRRAD_1,IRRAD_2,IRRAD_3,IRRAD_4,IRRAD_5,IRRAD_6,IRRAD_7,IRRAD_8
0,0.0,0.0,0.0,1345.180534,2808.997561,4272.814589,5736.631616,7200.448644,8664.265671
1,0.0,0.0,0.0,1219.350127,2600.870634,3982.391141,5363.911648,6745.432155,8126.952662
2,0.0,0.0,0.0,1286.667397,2641.293990,3995.920583,5350.547176,6705.173769,8059.800362
3,0.0,0.0,0.0,1262.185892,2690.576839,4118.967786,5547.358734,6975.749681,8404.140628
4,0.0,0.0,0.0,1276.231420,2666.667893,4057.104366,5447.540839,6837.977311,8228.413784
...,...,...,...,...,...,...,...,...,...
1821,0.0,0.0,0.0,879.350765,2355.330836,3831.310907,5307.290978,6783.271050,8259.251121
1822,0.0,0.0,0.0,829.867757,2292.550481,3755.233204,5217.915928,6680.598652,8143.281376
1823,0.0,0.0,0.0,595.133688,2065.109866,3535.086043,5005.062221,6475.038398,7945.014576
1824,0.0,0.0,0.0,886.378722,2351.992801,3817.606880,5283.220960,6748.835039,8214.449118


In [149]:
dirname = "predicted_weather"
    
df = pd.read_csv(os.path.join(dirname, "interval_data/irrad.csv"))
# irrad  = checkIrrad(irrad)

# def checkIrrad(df: pd.DataFrame)->pd.DataFrame:
mask = df<0
df.loc[mask] = 0 
mask = df>40000000
df.loc[mask] = 40000000-1
    # return df



KeyError: "None of [Index([('I', 'R', 'R', 'A', 'D', '_', '0'),\n       ('I', 'R', 'R', 'A', 'D', '_', '1'),\n       ('I', 'R', 'R', 'A', 'D', '_', '2'),\n       ('I', 'R', 'R', 'A', 'D', '_', '3'),\n       ('I', 'R', 'R', 'A', 'D', '_', '4'),\n       ('I', 'R', 'R', 'A', 'D', '_', '5'),\n       ('I', 'R', 'R', 'A', 'D', '_', '6'),\n       ('I', 'R', 'R', 'A', 'D', '_', '7'),\n       ('I', 'R', 'R', 'A', 'D', '_', '8')],\n      dtype='object')] are in the [index]"

In [104]:
pd.DataFrame(weather.export())

""
